## Apriori

In [1]:
import numpy as np
import pandas as pd
import copy
import sys
import psutil
import csv
from csv import reader
from itertools import combinations

### Frequent itemset generation of the Apriori algorithm

In [2]:
supp = lambda a, n: [True, a / n] if (a / n >= min_supp) else [False, a / n]

def ap_gen(T, min_supp):
    idx = 0
    frequent_subsets = []
    while True:
        idx += 1
        if idx == 1:
            frequent_items = []
            infrequent_items = []
            frequent_sets = []
            items, counts = get_unique_items(T)
            for i, c in zip(items, counts):
                if supp(c, len(T)):
                    frequent_items.append(i)
            frequent_sets.append(frequent_items)
        else:
            combs = list(combinations(frequent_items, idx))
            candidates = []
            for comb in combs:
                state = True
                for item in infrequent_items:
                    if len(np.intersect1d(comb, item)) == idx-1:
                        state = False
                if state:
                    candidates.append(comb)
            frequent_items = []
            infrequent_items = []
            for cand in candidates:
                count = [np.intersect1d(cand, t).shape[0] for t in T].count(len(cand))
                if supp(count, len(T))[0]:
                    frequent_items.append(cand)
                else:
                    infrequent_items.append(cand)
            if not frequent_items:
                break
            else:
                frequent_sets += frequent_items
            frequent_items, _ = get_unique_items(frequent_items)
    return frequent_sets

### Rule generation of the Apriori algorithm

In [5]:
def ap_genrules(f, H, min_supp, min_conf, writer):
    if H:
        if len(f) > len(H[0])+1:
            H_new = ap_gen(H, min_supp)
            if H_new == H:
                return
            for h in H_new:
                count_f = [np.intersect1d(f, t).shape[0] for t in T].count(len(f))
                support_f = supp(count_f, len(T))[1]
                f_h = list(set(copy.copy(f)).difference(set(h)))
                count_f_h = [np.intersect1d(f_h, t).shape[0] for t in T].count(len(f_h))
                support_f_h = supp(count_f_h, len(T))[1]
                confidence = support_f / support_f_h
                if confidence >= min_conf:
                    rules = [f_h, h, support_f_h, confidence, 
                                  confidence / support_f_h]
                    for i in range(len(rules)):
                        if rules[i] == rules[-1]:
                            rules[i] = str(np.round(rules[i], 3))
                        else:
                            rules[i] = str(rules[i])
                    write.writerow(rules)
                else:
                    H_new.remove(h)
            ap_genrules(f, H_new, min_supp, min_conf, writer)
        
        
def apriori(frequent_sets, min_supp, min_conf, writer):
    for f in frequent_sets[1:]:
        H = []
        A = []
        unique = np.unique(f)
        for u in unique:
            a = list(copy.copy(f))
            a.remove(u)
            A.append(a)        
        for a in A:
            count_f = [np.intersect1d(f, t).shape[0] for t in T].count(len(f))
            support_f = supp(count_f, len(T))[1]
            count_a = [np.intersect1d(a, t).shape[0] for t in T].count(len(a))
            confidence = support_f / supp(count_a, len(T))[1]
            if confidence >= min_conf:
                rule = set(copy.copy(f))
                rule = list(rule.difference(set(a)))
                count_rule = [np.intersect1d(rule, t).shape[0] for t in T].count(len(rule))
                support_rule = supp(count_rule, len(T))[1]
                rules = [a, rule, support_rule, confidence, 
                              confidence / support_rule]
                for i in range(len(rules)):
                    if rules[i] == rules[-1]:
                        rules[i] = str(np.round(rules[i], 3))
                    else:
                        rules[i] = str(rules[i])
                write.writerow(rules)
                H.append(rule)
        ap_genrules(f, H, min_supp, min_conf, write)

In [6]:
def load_data(filename):
    data = []
    with open(filename, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            data.append(row)
    return data

def to_df():
    df = pd.read_csv("rules.csv", names=["ru", "les", "support", "confidence", "lift"])
    rules = df["ru"].astype(str)+'->'+df["les"].astype(str)
    df.insert(0, "rules", rules)
    df = df.drop(["ru", "les"], axis=1)
    return df.sort_values(by="lift", ascending=False)

def get_unique_items(data):
    items = []
    for d in data:
        items += d
    return np.unique(items, return_counts=True)

### Testing algorithm on example dataset

In [12]:
T = load_data("example.csv")
min_supp = 0.4
min_conf = 0.8

frequent_sets = ap_gen(T, min_supp)
with open("rules.csv", 'w') as f:
    write = csv.writer(f)
    apriori(frequent_sets, min_supp, min_conf, write)    
to_df()

,rules,support,confidence,lift
6,"['a', 'd']->['e']",0.6,1.000000,1.667
7,"['a']->['d', 'e']",0.5,0.800000,1.600
1,['a']->['e'],0.6,0.800000,1.333
4,['e']->['d'],0.9,1.000000,1.111
5,"['a', 'e']->['d']",0.9,1.000000,1.111
8,"['b', 'e']->['d']",0.9,1.000000,1.111
2,['b']->['d'],0.9,0.857143,0.952
0,['a']->['d'],0.9,0.800000,0.889
3,['c']->['d'],0.9,0.800000,0.889


### Testing algorithm on "transactions.csv" (from presentation)

In [16]:
T = load_data("transactions.csv")
min_supp = 0.5
min_conf = 0.8

frequent_sets = ap_gen(T, min_supp)
with open("rules.csv", 'w') as f:
    write = csv.writer(f)
    apriori(frequent_sets, min_supp, min_conf, write)
    
to_df()

,rules,support,confidence,lift
0,['beer']->['diapers'],0.8,1.0,1.25
